In [9]:
import os
import sys
import json

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [10]:
def _load_feature_map_from_npz(feature_map_path):
    feature_map = np.load(feature_map_path)
    return feature_map['feature_map'], feature_map['label']

In [11]:
class my_str(str):
    def __lt__(a,b):
        a = a.split('.')[0]
        b = b.split('.')[0]
        prefix_a = '_'.join([a.split('_')[0], a.split('_')[1]])
        prefix_b = '_'.join([b.split('_')[0], b.split('_')[1]])
        if prefix_a == prefix_b:
            return int(a.split('_')[-1]) < int(b.split('_')[-1])
        else:
            return prefix_a < prefix_b
    def __gt__(a,b):        
        a = a.split('.')[0]
        b = b.split('.')[0]
        prefix_a = '_'.join([a.split('_')[0], a.split('_')[1]])
        prefix_b = '_'.join([b.split('_')[0], b.split('_')[1]])
        if prefix_a == prefix_b:
            return int(a.split('_')[-1]) < int(b.split('_')[-1])
        else:
            return prefix_a > prefix_b
    
def mfunc(x):
    return my_str(x)

In [21]:
a = map(mfunc, os.listdir('../data/feature_maps/validation'))
step_videos = 0
count = 0
window_size = 6
batch_size = 15
for item in sorted(a):
    images = labels = video_id = None
    if count == 60:
        break
    count += 1
    for idx in range(0, 15):
        feature_map_path = os.path.join('../data/feature_maps/validation', item)
        images, labels = _load_feature_map_from_npz(feature_map_path)
        video_id = item.split('.')[0]
        print('current fmap: ', video_id)
        batch_num = int(item.split('.')[0].split('_')[-1])
        video_name = item.split('.')[0].split('batch')[0][:-1]
        
        available_formats = ['.mkv', '.webm', '.mp4']
        for vformat in available_formats:
            video_path = os.path.join('../data/videos/' + 'validation', video_name + vformat)
            if os.path.isfile(video_path):
                break
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        metadata = {
            'video_id': video_name + vformat
        }
        cap.release()
        if idx + window_size > images.shape[0]:
            print('overlap')
            next_fmap_name = video_name + '_' + 'batch_' + str(batch_num + 1) + '.npz'
            print('next_fmap: ', next_fmap_name)
            if not os.path.isfile('../data/feature_maps/validation/' + next_fmap_name):
                break
            images_next, labels_next = _load_feature_map_from_npz(feature_map_path)
            offset = window_size - (images.shape[0] - idx)
            print('from: ', idx, 'to', images.shape[0] - 1, 'in', video_id)
            print('from: ', 0, 'to', offset - 1, 'in', next_fmap_name)
            print(images[idx:images.shape[0], :].shape, images_next[0:offset, :].shape)
            images = np.concatenate((images[idx:images.shape[0], :], images_next[0:offset, :]), axis=0)
            labels = np.concatenate((labels[idx:images.shape[0], :], labels_next[0:offset, :]), axis=0)
            frame_number_ini = (batch_size * (batch_num - 1) + idx) * 6
            frame_number_end = (batch_size * (batch_num) + offset - 1) * 6
            metadata['segment'] = [frame_number_ini / fps, frame_number_end / fps]
            
        else:
            print('no overlap')
            print('from: ', idx, 'to', idx + window_size - 1, 'in', video_id)
            images = images[idx:idx+window_size, :]
            labels = labels[idx:idx+window_size, :]
            images = np.reshape(images, (15, -1))
            frame_number_ini = (batch_size * (batch_num - 1) + idx) * 6
            frame_number_end = (batch_size * (batch_num - 1) + idx + window_size - 1) * 6
            metadata['segment'] = [frame_number_ini / fps, frame_number_end / fps]
            
        print(images.shape, labels.shape, metadata)
        print('*********************************')
        

current fmap:  -8awLlFLcQc_batch_1
no overlap
from:  0 to 5 in -8awLlFLcQc_batch_1
(15, 15360) (6, 34) {'video_id': '-8awLlFLcQc.mkv', 'segment': [0.0, 1.0010000000000001]}
*********************************
current fmap:  -8awLlFLcQc_batch_1
no overlap
from:  1 to 6 in -8awLlFLcQc_batch_1
(15, 15360) (6, 34) {'video_id': '-8awLlFLcQc.mkv', 'segment': [0.20020000000000002, 1.2012]}
*********************************
current fmap:  -8awLlFLcQc_batch_1
no overlap
from:  2 to 7 in -8awLlFLcQc_batch_1
(15, 15360) (6, 34) {'video_id': '-8awLlFLcQc.mkv', 'segment': [0.40040000000000003, 1.4014]}
*********************************
current fmap:  -8awLlFLcQc_batch_1
no overlap
from:  3 to 8 in -8awLlFLcQc_batch_1
(15, 15360) (6, 34) {'video_id': '-8awLlFLcQc.mkv', 'segment': [0.6006, 1.6016000000000001]}
*********************************
current fmap:  -8awLlFLcQc_batch_1
no overlap
from:  4 to 9 in -8awLlFLcQc_batch_1
(15, 15360) (6, 34) {'video_id': '-8awLlFLcQc.mkv', 'segment': [0.800800000000

In [22]:
os.path.join('..data', 'videos', 'validation')

'..data/videos/validation'